In [23]:
import cv2
from tracker import *
import torch
import numpy as np

# 1. Cargar el modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Definir clases de interés
CLASSES_OF_INTEREST = ['car']

# 2. Definir la clase Tracker (por centroides)
import numpy as np
from scipy.spatial.distance import cdist

class Tracker:
    MAX_DISAPPEAR_LIMIT = 3

    def __init__(self):
        self.next_unique_id = 0
        self.trackers = {}
        self.disappear_trackers = {}
        self.tracked_bboxes = {}
        self.previous_y = {}  # Añadir variable para almacenar la posición Y anterior

    def init_object(self, centroid, boxes):
        self.trackers[self.next_unique_id] = centroid
        self.tracked_bboxes[self.next_unique_id] = boxes
        self.disappear_trackers[self.next_unique_id] = 0
        self.previous_y[self.next_unique_id] = centroid[1]  # Guardar la posición Y inicial
        self.next_unique_id += 1

    def del_object(self, track_id):
        del self.trackers[track_id]
        del self.tracked_bboxes[track_id]
        del self.disappear_trackers[track_id]
        del self.previous_y[track_id]

    def update_object(self, bboxes):
        if len(bboxes) == 0:
            for oid in list(self.disappear_trackers.keys()):
                self.disappear_trackers[oid] += 1
                if self.disappear_trackers[oid] > Tracker.MAX_DISAPPEAR_LIMIT:
                    self.del_object(oid)
            return self.tracked_bboxes
        else:
            input_centroids = np.zeros((len(bboxes), 2))
            for i in range(len(bboxes)):
                x, y, w, h = bboxes[i][0], bboxes[i][1], bboxes[i][2], bboxes[i][3]
                cx, cy = x + w / 2, y + h / 2
                input_centroids[i] = (cx, cy)

            if len(self.trackers) == 0:
                for i in range(len(input_centroids)):
                    self.init_object(input_centroids[i], bboxes[i])
            else:
                tracker_centroids = list(self.trackers.values())
                distance_matrix = cdist(np.array(tracker_centroids), input_centroids)
                rows = distance_matrix.min(axis=1).argsort()
                cols = distance_matrix.argmin(axis=1)[rows]

                usedRows = set()
                usedCols = set()

                tracker_ids = list(self.trackers.keys())
                for row, col in zip(rows, cols):
                    if row in usedRows or col in usedCols:
                        continue
                    track_id = tracker_ids[row]
                    self.trackers[track_id] = input_centroids[col]
                    self.tracked_bboxes[track_id] = bboxes[col]
                    self.disappear_trackers[track_id] = 0
                    usedRows.add(row)
                    usedCols.add(col)

                unusedRows = set(range(0, distance_matrix.shape[0])).difference(usedRows)
                unusedCols = set(range(0, distance_matrix.shape[1])).difference(usedCols)

                if distance_matrix.shape[0] >= distance_matrix.shape[1]:
                    for r in unusedRows:
                        track_id = tracker_ids[r]
                        self.disappear_trackers[track_id] += 1
                        if self.disappear_trackers[track_id] > Tracker.MAX_DISAPPEAR_LIMIT:
                            self.del_object(track_id)
                else:
                    for c in unusedCols:
                        self.init_object(input_centroids[c], bboxes[c])

        return self.tracked_bboxes

# 3. Crear objeto de tracker
tracker2 = Tracker()

# Cargar el video
cap = cv2.VideoCapture("output2.mp4")

# Inicializar contadores de coches
car_up_count = 0
car_down_count = 0

# Definir el límite de la región a excluir (por ejemplo, excluye la parte derecha del ROI)
exclude_right_threshold = 200  # Ajusta este valor para definir la región a la derecha que quieres ignorar

# 4. Bucle sobre los frames del video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Extraer Región de interés (ROI)
    roi = frame[350:820, 100:520]

    # 1. Detección de objetos
    results = model(roi)

    detections = []
    for det in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = det

        if model.names[int(cls)] == 'car':
            w = int(x2 - x1)
            h = int(y2 - y1)
            x = int(x1)
            y = int(y1)

            # Filtrar los coches que están en la parte derecha, ignorando los aparcados
            if x + w < exclude_right_threshold:  # Si el coche está fuera de la región excluida
                detections.append([x, y, w, h])

    # 2. Actualizar el tracker
    boxes_ids = tracker2.update_object(detections)

    # Comparar la posición Y actual con la anterior y contar
    for box_id, box in boxes_ids.items():
        x, y, w, h = box
        id = box_id

        # Verificar si el ID ya ha sido registrado antes
        if id in tracker2.previous_y:
            if y < tracker2.previous_y[id]:  # Si Y disminuye, el coche está bajando
                car_down_count += 1
            elif y > tracker2.previous_y[id]:  # Si Y aumenta, el coche está subiendo
                car_up_count += 1

        # Actualizar la posición Y anterior
        tracker2.previous_y[id] = y

        # Dibujar resultados
        cv2.putText(roi, f"ID: {id}", (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Dibujar los contadores de coches
    cv2.putText(frame, f"Cars Up: {car_up_count}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Cars Down: {car_down_count}", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Mostrar resultados
    cv2.imshow("ROI", roi)
    cv2.imshow("Frame", frame)

    # Controlar los frames del video
    cap.grab()
    key = cv2.waitKey(30)
    # Presionar 'Esc' para salir
    if key == 27:
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


requirements: Ultralytics requirements ['pillow>=10.3.0', 'requests>=2.32.2', 'setuptools>=70.0.0'] not found, attempting AutoUpdate...


Using cache found in C:\Users\LAURA/.cache\torch\hub\ultralytics_yolov5_master


Retry 1/2 failed: Command 'pip install --no-cache-dir "pillow>=10.3.0" "requests>=2.32.2" "setuptools>=70.0.0" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "pillow>=10.3.0" "requests>=2.32.2" "setuptools>=70.0.0" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "pillow>=10.3.0" "requests>=2.32.2" "setuptools>=70.0.0" ' returned non-zero exit status 1.


YOLOv5  2024-10-22 Python-3.9.0 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


KeyboardInterrupt: 

In [24]:
# Liberar recursos
cap.release()
cv2.destroyAllWindows()